전자공시 OPEN DART 시스템

* https://opendart.fss.or.kr/

* [인증키 신청/관리]-[오픈API 이용현황]

In [1]:
import requests
import pandas as pd
from tqdm import tqdm

# 1. 고유번호 구하기

In [2]:
key = "d7ace37298ef7036447ceb35a4a09a683e963f4c"

In [3]:
url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={key}"

In [4]:
from urllib.request import urlretrieve

In [5]:
urlretrieve(url, "temp.zip")

('temp.zip', <http.client.HTTPMessage at 0x7f85d1b37040>)

In [6]:
from zipfile import ZipFile

In [7]:
ZipFile("./temp.zip").extractall()

In [8]:
corp_list = pd.read_xml("./CORPCODE.xml")

In [9]:
corp_list.head()

,corp_code,corp_name,stock_code,modify_date
0,434003,다코,NaN,20170630
1,434456,일산약품,NaN,20170630
2,430964,굿앤엘에스,NaN,20170630
3,432403,한라판지,NaN,20170630
4,388953,크레디피아제이십오차유동화전문회사,NaN,20170630


In [12]:
corp_list['corp_code'] = corp_list['corp_code'].astype(str).str.zfill(8)

In [13]:
corp_list = corp_list.dropna(subset=['stock_code'])

In [14]:
corp_list.head()

,corp_code,corp_name,stock_code,modify_date
2009,00260985,한빛네트,36720.0,20170630
2021,00264529,엔플렉스,40130.0,20170630
2022,00358545,동서정보기술,55000.0,20170630
2784,00231567,애드모바일,32600.0,20170630
3889,00247939,씨모스,37600.0,20170630


In [16]:
corp_list['stock_code'] = corp_list['stock_code'].astype(int).astype(str).str.zfill(6)

In [17]:
corp_list.head()

,corp_code,corp_name,stock_code,modify_date
2009,00260985,한빛네트,036720,20170630
2021,00264529,엔플렉스,040130,20170630
2022,00358545,동서정보기술,055000,20170630
2784,00231567,애드모바일,032600,20170630
3889,00247939,씨모스,037600,20170630


##1. 매출 총이익 구하기

* 매출 총이익 = 매출액 - 매출원가

In [18]:
corp_list[corp_list['corp_name']=='삼성전자']

,corp_code,corp_name,stock_code,modify_date
79106,00126380,삼성전자,005930,20220509


In [19]:
key = "d7ace37298ef7036447ceb35a4a09a683e963f4c"

In [20]:
params = {
    "crtfc_key" : key,
    "corp_code" : "00126380",
    "bsns_year" : 2020,
    "reprt_code" : 11011,
    "fs_div" : "OFS"
}

In [21]:
url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
response = requests.get(url, params=params)
data = response.json()

In [22]:
response.url

'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=d7ace37298ef7036447ceb35a4a09a683e963f4c&corp_code=00126380&bsns_year=2020&reprt_code=11011&fs_div=OFS'

In [23]:
data

{'status': '000',
 'message': '정상',
 'list': [{'rcept_no': '20210309000744',
   'reprt_code': '11011',
   'bsns_year': '2020',
   'corp_code': '00126380',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'account_id': 'ifrs-full_CurrentAssets',
   'account_nm': '유동자산',
   'account_detail': '-',
   'thstrm_nm': '제 52 기',
   'thstrm_amount': '73798549000000',
   'frmtrm_nm': '제 51 기',
   'frmtrm_amount': '72659080000000',
   'bfefrmtrm_nm': '제 50 기',
   'bfefrmtrm_amount': '80039455000000',
   'ord': '1',
   'currency': 'KRW'},
  {'rcept_no': '20210309000744',
   'reprt_code': '11011',
   'bsns_year': '2020',
   'corp_code': '00126380',
   'sj_div': 'BS',
   'sj_nm': '재무상태표',
   'account_id': 'ifrs-full_CashAndCashEquivalents',
   'account_nm': '현금및현금성자산',
   'account_detail': '-',
   'thstrm_nm': '제 52 기',
   'thstrm_amount': '989045000000',
   'frmtrm_nm': '제 51 기',
   'frmtrm_amount': '2081917000000',
   'bfefrmtrm_nm': '제 50 기',
   'bfefrmtrm_amount': '2607957000000',
   'ord': '2',
   'cu

In [24]:
df = pd.DataFrame(data['list'])

In [25]:
df.head()

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 52 기,73798549000000,제 51 기,72659080000000,제 50 기,80039455000000,1,KRW,NaN
1,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 52 기,989045000000,제 51 기,2081917000000,제 50 기,2607957000000,2,KRW,NaN
2,20210309000744,11011,2020,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 52 기,29101284000000,제 51 기,26501392000000,제 50 기,34113871000000,3,KRW,NaN
3,20210309000744,11011,2020,00126380,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,-,제 52 기,24736740000000,제 51 기,26255438000000,제 50 기,24933267000000,4,KRW,NaN
4,20210309000744,11011,2020,00126380,BS,재무상태표,dart_ShortTermOtherReceivables,미수금,-,제 52 기,1898583000000,제 51 기,2406795000000,제 50 기,1515079000000,5,KRW,NaN


In [26]:
df[df['sj_div']=='IS']['account_nm'].value_counts()

수익(매출액)            1
매출원가               1
매출총이익              1
판매비와관리비            1
영업이익               1
기타수익               1
기타비용               1
금융수익               1
금융비용               1
법인세비용차감전순이익(손실)    1
법인세비용              1
계속영업이익(손실)         1
당기순이익(손실)          1
기본주당이익(손실)         1
희석주당이익(손실)         1
Name: account_nm, dtype: int64

In [27]:
df[df['account_nm']=='수익(매출액)']

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
44,20210309000744,11011,2020,00126380,IS,손익계산서,ifrs-full_Revenue,수익(매출액),-,제 52 기,166311191000000,제 51 기,154772859000000,제 50 기,170381870000000,0,KRW,


In [28]:
df[df['account_id']=='ifrs-full_GrossProfit']['thstrm_amount'].values[0]

'49557772000000'

In [29]:
df[df['account_id']=='ifrs-full_Revenue']['thstrm_amount'].values[0]

'166311191000000'

##3. 다수 기업의 매출 총이익 구하기

In [30]:
# 매출 총이익 확인부분 변경

def get_sales_inc(corp_code, corp_name):
    key = "d7ace37298ef7036447ceb35a4a09a683e963f4c"

    params = {
    "crtfc_key" : key,
    "corp_code" : corp_code,
    "bsns_year" : 2020,
    "reprt_code" : 11011,
    "fs_div" : "OFS"
    }

    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    response = requests.get(url, params=params)
    data = response.json()

    try:
        df = pd.DataFrame(data['list'])
        sales_inc = df[df['account_id']=='ifrs-full_GrossProfit']['thstrm_amount'].values[0]
        sales = df[df['account_id']=='ifrs-full_Revenue']['thstrm_amount'].values[0]
        sales_inc_rate = int(sales_inc) / int(sales) *100
    except Exception as e:
        print(corp_code + ", "+ corp_name + ". " + data['message'] + ", " + str(e))
        return [corp_code, corp_name, None], data['message']

    return [corp_code, corp_name, sales_inc_rate], data['message']

코스피 상장 종목을 대상으로 매출 총이익 구하기

In [31]:
corp_list.head()

,corp_code,corp_name,stock_code,modify_date
2009,00260985,한빛네트,036720,20170630
2021,00264529,엔플렉스,040130,20170630
2022,00358545,동서정보기술,055000,20170630
2784,00231567,애드모바일,032600,20170630
3889,00247939,씨모스,037600,20170630


In [32]:
corp_list.dtypes

corp_code      object
corp_name      object
stock_code     object
modify_date     int64
dtype: object

In [33]:
corp_tar = corp_list[corp_list['modify_date'] > 20200000]

In [34]:
corp_tar.head()

,corp_code,corp_name,stock_code,modify_date
43719,01199985,미래에셋대우기업인수목적1호,265480,20200103
44363,00473400,한컴지엠디,077280,20200116
45046,01201581,한화수성기업인수목적,265920,20200115
46248,00527880,썬테크놀로지스,122800,20200212
46483,00117638,동양텔레콤,007150,20200226


In [35]:
len(corp_tar)

2841

In [36]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import FinanceDataReader as fdr

kospi = fdr.StockListing("KOSPI")

In [38]:
len(kospi)

943

In [39]:
kospi

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,55300,2,-1300,-2.30,56000,56200,55300,11295935,628653753150,330128975015000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,435500,2,-1000,-0.23,435500,442500,432500,265226,116156988000,101907000000000,234000000,STK
2,207940,KR7207940008,삼성바이오로직스,KOSPI,,821000,2,-9000,-1.08,822000,831000,820000,35179,28994143000,58433854000000,71174000,STK
3,000660,KR7000660001,SK하이닉스,KOSPI,,75000,2,-1000,-1.32,75200,75600,74700,1736661,130328025000,54600177375000,728002365,STK
4,051910,KR7051910008,LG화학,KOSPI,,600000,2,-2000,-0.33,597000,607000,593000,103968,62525555000,42355405800000,70592343,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,002785,KR7002781003,진흥기업우B,KOSPI,,4935,1,30,0.61,4890,4935,4860,1102,5377790,4230370830,857218,STK
939,001525,KR7001521004,동양우,KOSPI,,6390,2,-20,-0.31,6420,6420,6330,2119,13453380,3947301090,617731,STK
940,001527,KR7001522002,동양2우B,KOSPI,,12600,2,-300,-2.33,13800,13800,12600,1823,23006050,3887969400,308569,STK
941,002787,KR7002782001,진흥기업2우B,KOSPI,,13000,1,100,0.78,12700,13000,12700,185,2394800,3832504000,294808,STK


In [40]:
len(kospi.dropna())

943

In [41]:
kospi = kospi.dropna()

In [43]:
corp_tar = pd.merge(corp_tar, kospi.rename(columns={"Code" : "stock_code"}), how='inner', on='stock_code')

In [44]:
len(corp_tar)

825

In [45]:
corp_tar.head()

,corp_code,corp_name,stock_code,modify_date,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,00435297,맥쿼리인프라,088980,20200715,KR7088980008,맥쿼리인프라,KOSPI,,11050,2,-100,-0.90,11200,11200,11000,727090,8050312550,4473543294350,404845547,STK
1,01009789,코스맥스,192820,20211122,KR7192820009,코스맥스,KOSPI,,74100,1,800,1.09,73100,74100,71800,112038,8220366400,840998616900,11349509,STK
2,00102113,경인양행,012610,20211206,KR7012610002,경인양행,KOSPI,,4090,2,-110,-2.62,4155,4205,4060,75390,309361235,170210932850,41616365,STK
3,00129509,한익스프레스,014130,20211206,KR7014130009,한익스프레스,KOSPI,,5260,2,-80,-1.50,5270,5350,5160,12837,67209230,63120000000,12000000,STK
4,00210856,코아스,071950,20211126,KR7071950000,코아스,KOSPI,,730,2,-31,-4.07,760,760,715,157407,114313404,22562263300,30907210,STK


In [46]:
sale_inc_list = list()

for i in tqdm(corp_tar.iterrows()):
    try:
        temp_list, message = get_sales_inc(i[1]['corp_code'], i[1]['corp_name'])
        sale_inc_list.append(temp_list)
    except Exception as e:
        pass

1it [00:01,  1.10s/it]

00435297, 맥쿼리인프라. 조회된 데이타가 없습니다., 'list'


15it [00:18,  1.27s/it]

00148993, 하이트진로홀딩스. 정상, index 0 is out of bounds for axis 0 with size 0


21it [00:25,  1.09s/it]

...

807it [15:56,  1.11s/it]

00123772, 부국증권. 조회된 데이타가 없습니다., 'list'


819it [16:11,  1.23s/it]

00632304, JW홀딩스. 정상, index 0 is out of bounds for axis 0 with size 0


825it [16:18,  1.19s/it]


In [47]:
df = pd.DataFrame(sale_inc_list, columns=['corp_code', 'corp_name', 'sales_inc_rate'])

In [48]:
len(df)

825

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825 entries, 0 to 824
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   corp_code       825 non-null    object 
 1   corp_name       825 non-null    object 
 2   sales_inc_rate  662 non-null    float64
dtypes: float64(1), object(2)
memory usage: 19.5+ KB


In [50]:
df = df.dropna(subset=['sales_inc_rate'])

In [51]:
len(df)

662

In [52]:
df.dtypes

corp_code          object
corp_name          object
sales_inc_rate    float64
dtype: object

In [53]:
temp = df.sort_values('sales_inc_rate', ascending=False).head(50)

In [54]:
import plotly.express as px
px.bar(data_frame=temp, x='corp_name', y='sales_inc_rate').update_xaxes(categoryorder='total descending')